In [2]:
import sys
sys.path.append('../')

import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

In [3]:
df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_756_hour,rides_previous_755_hour,rides_previous_754_hour,rides_previous_753_hour,rides_previous_752_hour,rides_previous_751_hour,rides_previous_750_hour,rides_previous_749_hour,rides_previous_748_hour,rides_previous_747_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,0.0,0.0,5.0,4.0,3.0,4.0,3.0,2022-02-01 12:00:00,4,7.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,2.0,12.0,12.0,4.0,2.0,7.0,2022-02-02 12:00:00,4,5.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,0.0,0.0,11.0,13.0,10.0,5.0,5.0,2022-02-03 12:00:00,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,0.0,2.0,3.0,16.0,5.0,6.0,5.0,2022-02-04 12:00:00,4,4.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,1.0,3.0,0.0,2.0,5.0,4.0,12.0,2022-02-05 12:00:00,4,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27 12:00:00,199,0.0
87504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28 12:00:00,199,0.0
87505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29 12:00:00,199,0.0
87506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30 12:00:00,199,0.0


In [4]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(df, 
                                                    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
                                                    target_column_name='target_rides_next_hour')
print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(31440, 758)
y_train.shape=(31440,)
X_test.shape=(56068, 758)
y_test.shape=(56068,)


In [5]:
import lightgbm as lgb

In [6]:
# use only past rides data (excluding column = pickup_hour)
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]

In [7]:
model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.934094 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 172818
[LightGBM] [Info] Number of data points in the train set: 31440, number of used features: 756
[LightGBM] [Info] Start training from score 24.422646


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [8]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)

In [9]:
from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae:.4f}')

2.9736
